In [ ]:
 from google.colab import drive
 drive.mount('/content/drive')

In [ ]:
# download yolo
!git clone https://github.com/ultralytics/yolov5  # clone repo
!pip install -qr yolov5/requirements.txt # install dependencies (ignore errors)
# download my moedels
%cd yolov5/
!bash /content/yolov5/weights/download_weights.sh

In [ ]:
# Load my model for predicting traffic signs
from keras.models import load_model

new_model = load_model("/content/drive/My Drive/traffic sign recognition/traffic_sign_recognition.h5")
new_model.summary()

In [ ]:
# creating folders
%cd ..
!mkdir fromVideo
!mkdir makeVideo
!mkdir curFolder

In [4]:
# get every frame of my video

import cv2
vidcap = cv2.VideoCapture('/content/drive/MyDrive/tests/trafficLights.mp4')
success,image = vidcap.read()
count = 0
while success:
  cv2.imwrite("/content/fromVideo/frame%d.jpg" % count, image)     # save frame as JPEG file      
  success,image = vidcap.read()
  print('Read a new frame: ', success)
  count += 1

Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame

First we got to find and predict traffic signs

In [6]:
# get predicted images for traffic signs 
import os
from tqdm import tqdm

weightPath_traffic_signs = "/content/drive/My Drive/YOLO/costum Yolo/signs detection/yolo to traffiSignRec/best.pt"
myDir = "/content/fromVideo"

for img in tqdm(os.listdir(myDir), total=len(os.listdir(myDir))):
    myPath = os.path.join(myDir,img)
    output = !python /content/yolov5/detect.py --source "$myPath" --weights '$weightPath_traffic_signs' --save-txt --conf 0.3

100%|██████████| 90/90 [09:47<00:00,  6.53s/it]


In [7]:
# get img width and height
img = cv2.imread("/content/fromVideo/frame0.jpg")
imgHeight, imgWidth, channels = img.shape

In [8]:
# Preprocessing images

def blurImage(img):
    blur = cv2.GaussianBlur(img, (5, 5), 0)
    return img

def grayscale(img):
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    return img

def equalize(img):
    img = cv2.equalizeHist(img)
    return img


def preprocessing(img):
    img = blurImage(img) # No noise
    img = grayscale(img) # Convert to grayscale
    img = equalize(img)  # Standarize the lighting
    img = img/255        # Normalize values between 0-1
    return img

In [9]:
# Read CSV File for predicted labels
import pandas as pd
data = pd.read_csv("/content/drive/My Drive/traffic sign recognition/dataset/labels.csv")
print("data shape ",data.shape,type(data))
print(data)

data shape  (43, 2) <class 'pandas.core.frame.DataFrame'>
    ClassId                                               Name
0         0                               Speed limit (20km/h)
1         1                               Speed limit (30km/h)
2         2                               Speed limit (50km/h)
3         3                               Speed limit (60km/h)
4         4                               Speed limit (70km/h)
5         5                               Speed limit (80km/h)
6         6                        End of speed limit (80km/h)
7         7                              Speed limit (100km/h)
8         8                              Speed limit (120km/h)
9         9                                         No passing
10       10       No passing for vechiles over 3.5 metric tons
11       11              Right-of-way at the next intersection
12       12                                      Priority road
13       13                                              Yie

In [10]:
def getCalssName(classNo):
    return data.Name[classNo]

print(str(getCalssName(1)))

Speed limit (30km/h)


In [11]:
# Crop image function
def crop_image(x_center, y_center, width, height,img):
    # turn to integers
    x_center = int(x_center)
    y_center = int(y_center)
    width = int(width)
    height = int(height)
    # find the topX and topY
    topX = int(x_center - width/2)
    topY = int(y_center - height/2)
    #slice my image
    slicedImg = img[topY:topY + height, topX:topX + width]
    return slicedImg

In [12]:
# get predictions on sliced image
import numpy as np

def slicedImg_prediction(slicedImg):
    threshold = 0.75         # PROBABLITY THRESHOLD
    font = cv2.FONT_HERSHEY_COMPLEX_SMALL

    img = np.asarray(slicedImg)
    img = cv2.resize(img, (32, 32))
    img = preprocessing(img)
    img = img.reshape(1, 32, 32, 1)
    # cv2.putText(imgOrignal, "CLASS: " , (20, 35), font, 0.3, (255, 0, 0), 2, cv2.LINE_AA)
    # cv2.putText(imgOrignal, "PROBABILITY: ", (20, 75), font, 0.75, (255, 0, 0), 2, cv2.LINE_AA)
    # PREDICT IMAGE
    predictions = new_model.predict(img)
    classIndex = new_model.predict_classes(img)
    probabilityValue =np.amax(predictions)
    predClass = "no"
    
    if probabilityValue > threshold:
        predClass = str(getCalssName(classIndex[0]))
    return predClass
       

In [13]:
# write the class on image
def write_on_image(img, list_of_pred_classes):
    y = 0
    for pred_class in list_of_pred_classes:
        y = y+30
        cv2.putText(img, pred_class,
	                (10, y), cv2.FONT_HERSHEY_SIMPLEX,
	                1.0, (0, 255, 0), 3)
    return img

In [14]:
from pathlib import Path

detected_path = "/content/runs/detect"
for exps in tqdm(os.listdir(detected_path), total=len(os.listdir(detected_path))):
    expsPath = os.path.join(detected_path, exps) # /content/runs/detect/exp
    for expsInfo in os.listdir(expsPath):
        
        if expsInfo.endswith(".jpg"):
            test_image_path = os.path.join(expsPath, expsInfo) # /content/runs/detect/exp/frameX.jpg
            test_image = cv2.imread(test_image_path)
            image_for_cutting = cv2.imread(f"/content/fromVideo/{expsInfo}")

            cutLast4 = expsInfo.rsplit(".") # /content/runs/detect/exp/frameX
            # Check if labels exists
            labelsPath = f"{expsPath}/labels/{cutLast4[0]}.txt" # /content/runs/detect/exp/frameX.txt
            my_file = Path(labelsPath)
            if my_file.is_file():
                f = open(labelsPath, "r")
                fileslines = f.read().splitlines()
                counter = 0
                predicted_class = []
                for line in fileslines:
                    info = line.rsplit(" ")
                    clas = info[0]
                    x_center = int(float(info[1]) * imgWidth)
                    y_center = int(float(info[2]) * imgHeight)

                    mywidth = float(info[3]) * imgWidth
                    myheight = float(info[4]) * imgHeight
                    # crop image
                    slicedImg = crop_image(x_center,y_center,mywidth,myheight,image_for_cutting)
                    #check if sliced image exists
                    if slicedImg is not None:
                        #save image to a current folder
                        cv2.imwrite(f"/content/curFolder/{counter}.jpg", slicedImg)
                        counter = counter + 1
                        #take predicted class
                        predicted_class.append(slicedImg_prediction(slicedImg))
                # delete wrong predictions of yolo
                predicted_class = [i for i in predicted_class if i != "no"]
                # write the preddicted classes on image        
                test_image = write_on_image(test_image, predicted_class)
                
            #Save image
            cv2.imwrite(f"/content/makeVideo/{expsInfo}", test_image)
                    

  0%|          | 0/90 [00:00<?, ?it/s]

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


100%|██████████| 90/90 [00:17<00:00,  5.12it/s]


In [15]:
stop

Now the distances

In [16]:
# delete the runs folder from previus yolo
clearPath = "/content/runs"
!rm -r $clearPath

In [17]:
stop #gia to yolo5s douleyei mia xara to trexw sto l gia kltra apotelesmata

In [18]:

weightPath_coco = "/content/yolov5/yolov5l.pt"
myDir = "/content/makeVideo"

for img in tqdm(os.listdir(myDir), total=len(os.listdir(myDir))):
    myPath = os.path.join(myDir,img)
    output = !python /content/yolov5/detect.py --source "$myPath" --weights $weightPath_coco --save-txt --conf 0.3

100%|██████████| 90/90 [10:36<00:00,  7.07s/it]


In [19]:
# clear my make video file

directory = "/content/makeVideo"
for img in os.listdir(directory):
    curPath = f"{directory}/{img}"
    !rm $curPath

In [20]:
# take average widths

items_width = {
    "0":  0.4,   # person
    "2":  1.5,   # car
    "4":  0.5,   # motorbike
    "6":  2.1,   # bus
    "8":  2,     # truck
    "9":  0.3,   # traffic light
    "12": 0.6,   # stop sign
    "16": 0.3,   # cat
    "17": 0.4,   # dog
    "18": 1,     # horse
    "19": 0.9,   # sheep
    "20": 0.63   # cow 
}
print(items_width["0"])

0.4


In [21]:
# Initialize focal lenght
KNOWN_DISTANCE = 1
# initialize the known object width, which in this case, the piece of
# paper is 12 inches wide
KNOWN_WIDTH = 0.55
# load the furst image that contains an object that is KNOWN TO BE 2 feet
# from our camera, then find the paper marker in the image, and initialize
# the focal length

monitorWidth = 0.4 *imgWidth

# F = (P x  D) / W  # P: width in image
focalLength = (monitorWidth * KNOWN_DISTANCE) / KNOWN_WIDTH

In [22]:
def distance_to_camera(knownWidth, focalLength, perWidth):
	# compute and return the distance from the maker to the camera
	return (knownWidth * focalLength) / perWidth

In [23]:
from pathlib import Path

detected_path = "/content/runs/detect"
for exps in tqdm(os.listdir(detected_path), total=len(os.listdir(detected_path))):
    expsPath = os.path.join(detected_path, exps) # /content/runs/detect/exp
    for expsInfo in os.listdir(expsPath):
        
        if expsInfo.endswith(".jpg"):
            test_image_path = os.path.join(expsPath, expsInfo)
            test_image = cv2.imread(test_image_path)
            cutLast4 = expsInfo.rsplit(".")
            # Check if labels exists
            labelsPath = f"{expsPath}/labels/{cutLast4[0]}.txt"
            my_file = Path(labelsPath)
            if my_file.is_file():
                f = open(labelsPath, "r")
                fileslines = f.read().splitlines()
                
                for line in fileslines:
                    info = line.rsplit(" ")
                    clas = info[0]
                    x_center = int(float(info[1]) * imgWidth)
                    y_center = int(float(info[2]) * imgHeight)

                    mywidth = float(info[3]) * imgWidth

                    
                    # Check if Key Exists
                    if "{}".format(clas) in items_width:
                        known_width = items_width["{}".format(clas)]        
                        p = float(mywidth)
                        distance = distance_to_camera(known_width, focalLength, p)
                        
                        cv2.putText(test_image, "%.2fm" %distance,
	                        (x_center, y_center), cv2.FONT_HERSHEY_SIMPLEX,
	                        1.0, (0, 255, 0), 3)
                        
                        # Move every image with all predictions in another folder
            
            #Save image
            cv2.imwrite(f"/content/makeVideo/{expsInfo}", test_image)

100%|██████████| 90/90 [00:02<00:00, 33.65it/s]


In [24]:
stop

In [25]:
# make my video
fps = 20.0
image_folder = '/content/makeVideo'
video_name = 'All_together.mp4'
num_of_images = os.listdir(image_folder)
num_of_images = len(num_of_images)
images = []
for _ in range(0, num_of_images, 1):
    images.append(f"{image_folder}/frame{_}.jpg")


frame = cv2.imread(images[0])
height, width, layers = frame.shape

fourcc = cv2.VideoWriter_fourcc(*'MP4V')
video = cv2.VideoWriter(video_name, fourcc, fps, (width,height))

for image in images:
    video.write(cv2.imread(os.path.join(image_folder, image)))

cv2.destroyAllWindows()
video.release()